In [ ]:
#Import Required Libraries
import numpy as np # linear algebra
import keras # ML Model library
import numpy
import pandas as pd # Data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

#Pre-run configuration
np.random.seed(56)

#Model Configuration
image_width, image_height = 28, 28
batch_size = 300
no_epochs = 30
no_classes = 10
validation_split = 0.2
verbosity = 1
base_filters = 86


In [ ]:
#Import Training and Test datasets
train = pd.read_csv("../input/digit-recognizer/train.csv")
finaltest = pd.read_csv("../input/digit-recognizer/test.csv")
train['label'].value_counts(normalize=False)

In [ ]:
#Prepare Datalists
traindatalist = []
traindatalabels = []
y_trainlist = []
#y_trainstack =
y_trainlabels = []
x_trainlist = []
#x_trainstack = 
x_trainlabels = []
#x_train =
#x_test =
#y_train =
#y_test =

In [ ]:
#Prepare main training list and convert labels to binary
for i in range(no_classes):
    traindatalist.append(train)
    traindatalabels.append(int(i))
trainstack = pd.concat(traindatalist, keys=traindatalabels, axis=0)
del train
del traindatalist
del traindatalabels

In [ ]:
trainstack

In [ ]:
#Convert the label column to binary depending on what number is represented
for i in range(no_classes):
    #trainlist[i].at[(trainlist[i].label == int(i))] = 1
    trainstack.loc[(i,slice(None)),'label'] = np.where((trainstack.loc[(i,slice(None)),'label']==i), 1, 0)
    #trainlist[i].at[(trainlist[i].label != int(i))] = 0
   
    

In [ ]:
trainstack.loc[(5,slice(None))]

In [ ]:
trainstack.loc[(slice(None),slice(None)),'label']

In [ ]:
trainstack.loc[(0,slice(None)),'label'].value_counts()

In [ ]:
#x_trainstack.loc[(i,slice(None))].values.reshape(-1, image_width, image_height, 1)

In [ ]:
#Convert data to correct format for training
y_trainstack = trainstack.filter(['label'])
x_trainstack = trainstack.drop(['label'], axis = 1)
#del trainstack


In [ ]:
x_trainstack.loc[(1,slice(None))]

In [ ]:
#Reshape x_train data to (10, 42000, 28, 28)

## TODO Fix data reshaping
#for i in range(no_classes):
#    x_trainstack.loc[(i,slice(None))] = x_trainstack.loc[(i,slice(None))].values.reshape(42000, image_width, image_height)
#x_trainstack = x_trainstack.values.reshape((:, :, image_width, image_height))
for i in range(no_classes):
     x_trainstack.loc[(i,slice(None))] = x_trainstack.loc[(i,slice(None))].values.reshape(x_trainstack.loc[(i,slice(None))].shape[0],image_width, image_height, 1)
#    x_trainlist[i] = x_trainlist[i].values.reshape(x_trainlist[i].shape[0], image_width, image_height, 1)
#    x_trainlist[i] = x_trainlist[i].astype('float32')
#
#for i in range(len(x_trainlist)):
#    x_trainlist[i] = x_trainlist[i]/255
#    
#for i in range(len(x_trainlist)):
#    x_train[i], x_test[i], y_train[i], y_test[i] = train_test_split(x_trainlist[i], y_trainlist[i], test_size = validation_split)
x_train, x_test, y_train, y_test = train_test_split(x_trainstack, y_trainstack, test_size = validation_split)

In [ ]:
#Verify the data shape
x_train.shape

In [ ]:
#Verify label shape
y_train.shape

In [ ]:
#Define the models
models = list()
for i in range(no_classes):
    model = Sequential()
    model.add(Conv2D(filters = (base_filters/2), kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (image_width, image_height, 1)))
    model.add(Conv2D(filters = (base_filters/2), kernel_size = (5,5), padding = 'Same', activation = 'relu'))
    model.add(Conv2D(filters = (base_filters/2), kernel_size = (5,5), padding = 'Same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = (base_filters), kernel_size = (3,3), padding = 'Same', activation = 'relu'))
    model.add(Conv2D(filters = (base_filters), kernel_size = (3,3), padding = 'Same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = "softmax"))
    models.insert(i, model)
models[0].summary()

In [ ]:
#Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
#Compile the models
for i in range(len(models)):
    models[i].compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
#Set LRR
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3, verbose = 1, factor = 0.5, min_lr = 0.00001)

In [ ]:
x_train.loc[(i,slice(None))]

In [ ]:
#train the individual models for each number
histories = list()
for i in range(len(models)):
    histories.insert(i, models[i].fit(x_train.loc[(i,slice(None))], y_train.loc[(i,slice(None))], batch_size = batch_size, epochs = no_epochs,verbose = 1, validation_data = (x_test.loc[(i,slice(None))], y_test.loc[(i,slice(None))]), callbacks = [learning_rate_reduction]))